In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import ternary as te

In [ ]:
# Load wt. frequency data
allFreq = np.loadtxt("2021data.txt", skiprows=2, usecols=np.arange(1, 10)).transpose()

# Load grain sizes (in millimeters)
grainSizesData = np.loadtxt('2021data.txt', skiprows=1, usecols=np.arange(0, 9), dtype='str')[0]

# Create grain size labels and replace '<0.063' or 'pan' with '0.062'
grainSizes = list(grainSizesData[:-1])
grainSizes.append('0.062')

# Load sample labels
allLabels = np.loadtxt("2021data.txt", skiprows=2, usecols=0, dtype='str')

# Create DataFrame from the loaded data
df = pd.DataFrame(data=allFreq, columns=allLabels)
df.index = grainSizes

# Transpose the DataFrame to have samples as rows and grain sizes as columns
df = df.transpose()

# Reset index to turn the index into a column
df.reset_index(inplace=True)

# Rename the first column to "Sample ID"
df.rename(columns={'index': 'Sample ID'}, inplace=True)

# Convert grain size labels to numerical values
grain_sizes_numeric = [float(label) for label in grainSizes]

# Identify columns for each particle size category
gravel_columns = [df.columns[i+1] for i, size in enumerate(grain_sizes_numeric) if size >= 2]
sand_columns = [df.columns[i+1] for i, size in enumerate(grain_sizes_numeric) if 0.063 <= size < 2]
mud_column = [df.columns[i+1] for i, size in enumerate(grain_sizes_numeric) if size < 0.063]

# Calculate total sum for each row
df['Total'] = df[gravel_columns + sand_columns + mud_column].sum(axis=1)

# Calculate percentages for each category
df['Gravel percent'] = df[gravel_columns].sum(axis=1) / df['Total'] * 100
df['Sand percent'] = df[sand_columns].sum(axis=1) / df['Total'] * 100
df['Mud percent'] = df[mud_column].sum(axis=1) / df['Total'] * 100

# Drop the 'Total' column as it was only needed for the calculation
df.drop(columns=['Total'], inplace=True)

# Display the DataFrame
print(df)


In [ ]:
# Extract the percentage columns in the desired order for python-ternary and convert to NumPy array
percentages_array = df[['Sand percent', 'Gravel percent', 'Mud percent']].to_numpy()

# Display the NumPy array
print(percentages_array)

In [ ]:
#Creating a ternary diagram using python-ternary

def Ternary(array, scale=100, show_labels=True):
    figure, Texture = te.figure(scale=scale)
    figure.set_size_inches(16, 14)
    
    # Plot data 
    Texture.scatter(percentages_array, marker='o', c='black')
    
    # Draw Boundary and Gridlines
    Texture.boundary(linewidth=2.0)
    Texture.gridlines(color="blue", multiple=5)
    
    # Set Axis labels
    fontsize = 12
    offset = 0.2
    Texture.right_corner_label("Sand", fontsize=fontsize, weight='bold', offset=0)
    Texture.top_corner_label("Gravel", fontsize=fontsize, weight='bold', offset=0.2)
    Texture.left_corner_label("Silt and clay", fontsize=fontsize, weight='bold', offset=0)
    
    #This is to configure the style of the axes and ticks and specify their orientation/sense
    Texture.ticks(axis='lbr', multiple=5, linewidth=1, offset=0.025, tick_formats="%.0f", clockwise=False)
    Texture.get_axes().axis('off')
    Texture.clear_matplotlib_ticks()
    
    if show_labels:
        # add labels to each point
        label_offset = (5, 5)
        label_fontsize = 12
        labels=[]
        for i, txt in enumerate(df['Sample ID']):
            # Get the last four digits of the Sample ID using string slicing
            sample_id = str(txt)[-3:]
            #Annotate the point with the modified Sample ID
            Texture.annotate(sample_id, (df['Sand percent'][i], df['Gravel percent'][i], df["Mud percent"]), xytext=label_offset, textcoords='offset points', fontsize=label_fontsize)   
            
    Texture.show()

In [ ]:
Ternary(percentages_array, show_labels=True)

In [ ]:
#Another option using plotly

import plotly.express as px

fig = px.scatter_ternary(df, a="Gravel percent", b="Mud percent", c="Sand percent")

fig.show()

In [ ]:
#A more simple ternary diagram

import mpl_ternary as mplt #from https://marcoalopez.github.io/ternary_plots/

In [ ]:
# Extract the percentage columns for mpl_ternary and convert to NumPy array
percentages_array_normal = df[['Gravel percent', 'Sand percent', 'Mud percent']].to_numpy()


In [ ]:
# Normalize the data to lie between 0 and 1
x, y = mplt.tri2cart(upper_apex=(percentages_array_normal[:,0]/100),
                     right_apex=(percentages_array_normal[:,1]/100),
                     left_apex=(percentages_array_normal[:,2]/100))

# make the plot
fig, ax = mplt.ternary(upper_label='Gravel',
                       left_label='Mud',
                       right_label='Sand')

ax.plot(x, y, 'o', color='green', markersize=6)